In [5]:
import time
from collections import defaultdict, deque
import copy

In [6]:
class IndiaMapColoring:
    def __init__(self):
        self.states = [
            'AP', 'AR', 'AS', 'BR', 'CG', 'GA', 'GJ', 'HR', 'HP', 'JH',
            'KA', 'KL', 'MP', 'MH', 'MN', 'ML', 'MZ', 'NL', 'OD', 'PB',
            'RJ', 'SK', 'TN', 'TS', 'TR', 'UP', 'UK', 'WB', 'JK'
        ]
        self.colors = ['Red', 'Green', 'Blue', 'Yellow']
        self.adjacency = {
            'AP': ['TS', 'OD', 'TN', 'KA'],
            'AR': ['AS', 'NL'],
            'AS': ['AR', 'NL', 'ML', 'TR', 'MZ', 'MN', 'WB'],
            'BR': ['UP', 'JH', 'WB'],
            'CG': ['UP', 'JH', 'OD', 'MH'],
            'GA': ['MH', 'KA'],
            'GJ': ['MH', 'RJ'],
            'HR': ['PB', 'HP', 'UK', 'RJ'],
            'HP': ['JK', 'PB', 'HR', 'UK'],
            'JH': ['BR', 'UP', 'CG', 'OD', 'WB'],
            'KA': ['MH', 'AP', 'TS', 'TN', 'KL', 'GA'],
            'KL': ['KA', 'TN'],
            'MP': ['RJ', 'UP', 'CG', 'MH', 'GJ'],
            'MH': ['GJ', 'MP', 'CG', 'TS', 'KA', 'GA'],
            'MN': ['AS', 'MZ', 'NL'],
            'ML': ['AS', 'TR'],
            'MZ': ['AS', 'MN', 'TR'],
            'NL': ['AR', 'AS', 'MN'],
            'OD': ['WB', 'JH', 'CG', 'AP', 'TS'],
            'PB': ['JK', 'HP', 'HR', 'RJ'],
            'RJ': ['PB', 'HR', 'MP', 'GJ', 'UP'],
            'SK': ['WB'],
            'TN': ['AP', 'KA', 'KL'],
            'TS': ['MH', 'KA', 'AP', 'OD'],
            'TR': ['AS', 'ML', 'MZ'],
            'UP': ['UK', 'HR', 'RJ', 'MP', 'CG', 'JH', 'BR'],
            'UK': ['HP', 'HR', 'UP'],
            'WB': ['BR', 'JH', 'OD', 'AS', 'SK'],
            'JK': ['HP', 'PB']
        }
        self.domains = {state: self.colors[:] for state in self.states}
        self.plain_steps = 0
        self.heuristic_steps = 0
        self.ac3_steps = 0

    def is_safe(self, assignment, state, color):
        for neighbor in self.adjacency[state]:
            if neighbor in assignment and assignment[neighbor] == color:
                return False
        return True

    def plain_backtracking(self, assignment, unassigned_states):
        self.plain_steps += 1

        if not unassigned_states:
            return assignment

        state = unassigned_states[0]
        remaining_states = unassigned_states[1:]

        for color in self.colors:
            if self.is_safe(assignment, state, color):
                assignment[state] = color
                result = self.plain_backtracking(assignment, remaining_states)
                if result is not None:
                    return result
                del assignment[state]

        return None

    def get_mrv_variable(self, assignment, domains):
        unassigned = [state for state in self.states if state not in assignment]
        if not unassigned:
            return None

        min_values = float('inf')
        mrv_var = None

        for state in unassigned:
            if len(domains[state]) < min_values:
                min_values = len(domains[state])
                mrv_var = state

        return mrv_var

    def get_lcv_order(self, state, domains, assignment):
        color_constraints = []

        for color in domains[state]:
            if self.is_safe(assignment, state, color):
                constraints = 0
                for neighbor in self.adjacency[state]:
                    if neighbor not in assignment and color in domains[neighbor]:
                        constraints += 1
                color_constraints.append((color, constraints))

        color_constraints.sort(key=lambda x: x[1])
        return [color for color, _ in color_constraints]

    def update_domains_after_assignment(self, state, color, domains):
        new_domains = copy.deepcopy(domains)
        new_domains[state] = [color]

        for neighbor in self.adjacency[state]:
            if color in new_domains[neighbor]:
                new_domains[neighbor].remove(color)

        return new_domains

    def backtracking_with_heuristics(self, assignment, domains, debug=False):
        self.heuristic_steps += 1

        if len(assignment) == len(self.states):
            return assignment

        state = self.get_mrv_variable(assignment, domains)
        if state is None:
            return assignment

        if debug:
            print(f"MRV selected state: {state} (remaining values: {domains[state]})")

        colors_ordered = self.get_lcv_order(state, domains, assignment)

        if debug:
            print(f"LCV color order for {state}: {colors_ordered}")

        for color in colors_ordered:
            if self.is_safe(assignment, state, color):
                assignment[state] = color
                new_domains = self.update_domains_after_assignment(state, color, domains)

                result = self.backtracking_with_heuristics(assignment, new_domains, debug)
                if result is not None:
                    return result

                del assignment[state]

        return None

    def ac3(self, domains):
        queue = deque()

        for state in self.states:
            for neighbor in self.adjacency[state]:
                queue.append((state, neighbor))

        while queue:
            xi, xj = queue.popleft()

            if self.revise(domains, xi, xj):
                if not domains[xi]:
                    return False  # No solution possible

                for neighbor in self.adjacency[xi]:
                    if neighbor != xj:
                        queue.append((neighbor, xi))

        return True

    def revise(self, domains, xi, xj):
        revised = False
        to_remove = []

        for color_i in domains[xi]:
            satisfies = False
            for color_j in domains[xj]:
                if color_i != color_j:
                    satisfies = True
                    break

            if not satisfies:
                to_remove.append(color_i)
                revised = True

        for color in to_remove:
            domains[xi].remove(color)

        return revised

    def backtracking_with_ac3(self, assignment, domains, debug=False):
        self.ac3_steps += 1

        if len(assignment) == len(self.states):
            return assignment

        state = self.get_mrv_variable(assignment, domains)
        if state is None:
            return assignment

        if debug:
            print(f"AC-3+MRV selected state: {state} (remaining values: {domains[state]})")

        colors_ordered = self.get_lcv_order(state, domains, assignment)

        if debug:
            print(f"AC-3+LCV color order for {state}: {colors_ordered}")

        for color in colors_ordered:
            if self.is_safe(assignment, state, color):
                assignment[state] = color
                new_domains = self.update_domains_after_assignment(state, color, domains)

                if self.ac3(new_domains):
                    result = self.backtracking_with_ac3(assignment, new_domains, debug)
                    if result is not None:
                        return result

                del assignment[state]

        return None

    def solve_all_methods(self, debug=False):
        results = {}

        print("=" * 60)
        print("INDIA MAP COLORING PROBLEM - SOLUTION COMPARISON")
        print("=" * 60)

        print("\n1. PLAIN BACKTRACKING")
        print("-" * 30)
        self.plain_steps = 0
        start_time = time.time()

        assignment = {}
        solution1 = self.plain_backtracking(assignment, self.states[:])

        plain_time = time.time() - start_time
        results['Plain Backtracking'] = {
            'solution': solution1,
            'steps': self.plain_steps,
            'time': plain_time
        }

        print(f"Steps explored: {self.plain_steps}")
        print(f"Time taken: {plain_time:.4f} seconds")
        if solution1:
            print("Solution found:", solution1)


        print("\n2. BACKTRACKING WITH MRV + LCV HEURISTICS")
        print("-" * 45)
        self.heuristic_steps = 0
        start_time = time.time()

        assignment = {}
        domains = {state: self.colors[:] for state in self.states}
        solution2 = self.backtracking_with_heuristics(assignment, domains, debug)

        heuristic_time = time.time() - start_time
        results['Heuristic Backtracking'] = {
            'solution': solution2,
            'steps': self.heuristic_steps,
            'time': heuristic_time
        }

        print(f"Steps explored: {self.heuristic_steps}")
        print(f"Time taken: {heuristic_time:.4f} seconds")
        if solution2:
            print("Solution found:", solution2)


        print("\n3. BACKTRACKING WITH AC-3 + MRV + LCV")
        print("-" * 40)
        self.ac3_steps = 0
        start_time = time.time()

        assignment = {}
        domains = {state: self.colors[:] for state in self.states}
        solution3 = self.backtracking_with_ac3(assignment, domains, debug)

        ac3_time = time.time() - start_time
        results['AC-3 Backtracking'] = {
            'solution': solution3,
            'steps': self.ac3_steps,
            'time': ac3_time
        }

        print(f"Steps explored: {self.ac3_steps}")
        print(f"Time taken: {ac3_time:.4f} seconds")
        if solution3:
            print("Solution found:", solution3)


        print("\n" + "=" * 60)
        print("PERFORMANCE COMPARISON")
        print("=" * 60)
        print(f"{'Method':<25} {'Steps':<10} {'Time (s)':<12} {'Solution Found'}")
        print("-" * 60)

        for method, result in results.items():
            solution_status = "Yes" if result['solution'] else "No"
            print(f"{method:<25} {result['steps']:<10} {result['time']:<12.4f} {solution_status}")


        print("\n" + "=" * 60)
        print("ANALYSIS")
        print("=" * 60)

        if results['Plain Backtracking']['steps'] > 0:
            heuristic_improvement = ((results['Plain Backtracking']['steps'] - results['Heuristic Backtracking']['steps']) / results['Plain Backtracking']['steps']) * 100
            ac3_improvement = ((results['Plain Backtracking']['steps'] - results['AC-3 Backtracking']['steps']) / results['Plain Backtracking']['steps']) * 100

            print(f"Heuristic improvement: {heuristic_improvement:.1f}% reduction in steps")
            print(f"AC-3 improvement: {ac3_improvement:.1f}% reduction in steps")

            print("\nWhy AC-3 helps:")
            print("- Early detection of inconsistencies prevents exploring invalid branches")
            print("- Domain reduction through constraint propagation")
            print("- Combines with MRV/LCV for more efficient variable and value selection")

        return results

    def validate_solution(self, solution):
        if not solution:
            return False

        for state in solution:
            for neighbor in self.adjacency[state]:
                if neighbor in solution and solution[state] == solution[neighbor]:
                    print(f"Constraint violation: {state} and {neighbor} both have color {solution[state]}")
                    return False

        print("✓ Solution is valid - no adjacent states have the same color")
        return True

In [7]:
def main():
    solver = IndiaMapColoring()

    print("Running India Map Coloring Problem Solver...")
    print("States to color:", len(solver.states))
    print("Available colors:", solver.colors)
    print("Adjacency constraints:", sum(len(neighbors) for neighbors in solver.adjacency.values()) // 2, "edges")

    results = solver.solve_all_methods(debug=False)

    print("\n" + "=" * 60)
    print("SOLUTION VALIDATION")
    print("=" * 60)

    for method, result in results.items():
        print(f"\n{method}:")
        if result['solution']:
            solver.validate_solution(result['solution'])
        else:
            print("No solution found")

In [8]:
if __name__ == "__main__":
    main()

Running India Map Coloring Problem Solver...
States to color: 29
Available colors: ['Red', 'Green', 'Blue', 'Yellow']
Adjacency constraints: 54 edges
INDIA MAP COLORING PROBLEM - SOLUTION COMPARISON

1. PLAIN BACKTRACKING
------------------------------
Steps explored: 30
Time taken: 0.0001 seconds
Solution found: {'AP': 'Red', 'AR': 'Red', 'AS': 'Green', 'BR': 'Red', 'CG': 'Red', 'GA': 'Red', 'GJ': 'Red', 'HR': 'Red', 'HP': 'Green', 'JH': 'Green', 'KA': 'Green', 'KL': 'Red', 'MP': 'Green', 'MH': 'Blue', 'MN': 'Red', 'ML': 'Red', 'MZ': 'Blue', 'NL': 'Blue', 'OD': 'Blue', 'PB': 'Blue', 'RJ': 'Yellow', 'SK': 'Red', 'TN': 'Blue', 'TS': 'Yellow', 'TR': 'Yellow', 'UP': 'Blue', 'UK': 'Yellow', 'WB': 'Yellow', 'JK': 'Red'}

2. BACKTRACKING WITH MRV + LCV HEURISTICS
---------------------------------------------
Steps explored: 30
Time taken: 0.0025 seconds
Solution found: {'AP': 'Red', 'KA': 'Green', 'TN': 'Blue', 'KL': 'Red', 'TS': 'Blue', 'MH': 'Red', 'GA': 'Blue', 'OD': 'Green', 'CG': 'Blue'